<a href="https://colab.research.google.com/github/alik604/cyber-security/blob/master/Intrusion-Detection/KDD%20cup%20'99/Intrusion%20Detection%20V2%2099.99acc%20Multiclass_CLF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intrusion Detection 


## I do NOT use Kaggle. up to data code can be found @ https://github.com/alik604/cyber-security/tree/master/Intrusion-Detection

In [ ]:
%%capture

%config IPCompleter.greedy=True
import pandas as pd
# import seaborn as sns
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

!pip install catboost
import xgboost  # xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBClassifier
import lightgbm # lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html
import catboost # catboost.ai/docs/concepts/python-quickstart.html



import matplotlib.pyplot as plt
%matplotlib inline

from collections import Counter

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings("ignore")

In [ ]:
%%bash
wget "https://github.com/alik604/cyber-security/raw/master/Intrusion-Detection/KDD%20cup%20'99/kddcup.data.7z"
# repeat after me, GitHub is a Content Delivery Network

sudo apt-get install p7zip-full
7z x kddcup.data.7z

ls

In [ ]:
columns = ["duration" , "protocol_type" , "service" , "flag" , "src_bytes" , "dst_bytes" , "land" , "wrong_fragment" , "urgent" , "hot" , "num_failed_logins" , "logged_in" , "num_compromised" ,
           "root_shell" , "su_attempted" , "num_root" , "num_file_creations" , "num_shells" , "num_access_files" , "num_outbound_cmds" , "is_host_login" , "is_guest_login" , "count" , "srv_count" ,
           "serror_rate" , "srv_serror_rate" , "rerror_rate" , "srv_rerror_rate" , "same_srv_rate" , "diff_srv_rate" , "srv_diff_host_rate" , "dst_host_count" , "dst_host_srv_count" , "dst_host_same_srv_rate" ,
           "dst_host_diff_srv_rate" , "dst_host_same_src_port_rate" , "dst_host_srv_diff_host_rate" , "dst_host_serror_rate" , "dst_host_srv_serror_rate" , "dst_host_rerror_rate" , "dst_host_srv_rerror_rate" , "xAttack"]

all = pd.read_csv('./kddcup.data.csv', names=columns, index_col=0)

all.head(5)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(all.drop(['xAttack'], axis=1), all['xAttack'], test_size=0.66, random_state=42)

### Reduce train size for faster trainin, remove when in production

In [ ]:
# X_train=X_train.sample(frac =.50,random_state=1) # TODO. now this is uneeded as I used `train_test_split` above 

# X_train = train.drop('xAttack', axis=1)
# Y_train = train.loc[:,['xAttack']]
# X_test = test.drop('xAttack', axis=1)
# Y_test = test.loc[:,['xAttack']]

### The following few cells are taken from the 'sample code'

In [ ]:
le = preprocessing.LabelEncoder()

print(set(list(y_train)))
print(Counter(all['xAttack']))

y_train = le.fit_transform(y_train)
y_test  = le.fit_transform(y_test)


for col in ['protocol_type', 'service', 'flag']:
  X_train[col] = le.fit_transform(X_train[col])
  X_test[col] = le.fit_transform(X_test[col])

X_train.head(5)

## Drop features with lowest corr

In [ ]:
# print('\n')
# tmp = all.corr().abs()#.sort_values('xAttack')
# garbage = all['xAttack'].nsmallest(5)
# garbage
# garbage = list(garbage.index) + ['num_outbound_cmds','is_host_login']

## Drop features with lowest STD

In [ ]:
# df.ix[2,:]==0 # see where STD== 0, which means all values are the same... which is useless 

# con_list = [
#     'protocol_type', 'service', 'flag', 'land', 'logged_in', 'su_attempted',
#     'is_host_login', 'is_guest_login'
# ]

# df = X_train.drop(con_list, axis=1)

# #drop n smallest std features
df = X_train.std(axis=0).to_frame()
tmp = df.nsmallest(5, columns=0)
tmp = tmp.index.to_list() # list(tmp.transpose().columns)
tmp

# print(set(tmp + garbage))
# print(to_drop, len(to_drop))

In [ ]:
from sklearn import linear_model

from sklearn.ensemble import VotingClassifier

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import IsolationForest

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [ ]:
LR = linear_model.LinearRegression()
LR.fit(X_train, y_train)
lr_score = LR.score(X_test, y_test)
print('Linear regression processing ,,,')
print('Linear regression Score: %.2f %%' % lr_score)

In [ ]:
## to remove low Corr & low STD 

# try:  #TODO
#     X_train = X_train.drop(tmp,axis=1)
#     X_test = X_test.drop(tmp,axis=1)
# except:
#     None
    
# X_train.shape
# X_test.shape

In [ ]:
AB = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth = 3), n_estimators=50)
RF = RandomForestClassifier(n_estimators=100, criterion='entropy')
ET = ExtraTreesClassifier(n_estimators=100, criterion='gini', bootstrap=False)
GB = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=200, max_features='auto')

In [ ]:
_ = AB.fit(X_train[:1000], y_train[:1000])
AB_feature = AB.feature_importances_
ab_score = AB.score(X_test, y_test)

print('AdaBoostClassifier processing...')
print('AdaBoostClassifier Score: %.3f %%' % ab_score)

In [ ]:
_ = RF.fit(X_train, y_train)
RF_feature = RF.feature_importances_
rf_score = RF.score(X_test, y_test)

print('RandomForestClassifier processing...')
print('RandomForestClassifier Score: %.3f %%' % rf_score)

In [ ]:
_ = ET.fit(X_train, y_train)
ET_feature = ET.feature_importances_
et_score = ET.score(X_test, y_test)

print('ExtraTreesClassifier processing...')
print('ExtraTreeClassifier: %.3f %%' % et_score)

In [ ]:
_ = GB.fit(X_train[:2000], y_train[:2000])
GB_feature = GB.feature_importances_
gb_score = GB.score(X_test, y_test)

print('GradientBoostingClassifier processing...')
print('GradientBoostingClassifier Score: %.3f %%' % gb_score)

In [ ]:
feature_df = pd.DataFrame({'features': X_train.columns.values, # names
                           'AdaBoost' : AB_feature,
                           'RandomForest' : RF_feature,
                           'ExtraTree' : ET_feature,
                           'GradientBoost' : GB_feature
                          })
#feature_df.features
feature_df.head(10)

In [ ]:
n = 5
a_f = feature_df.nlargest(n, 'AdaBoost')
e_f = feature_df.nlargest(n, 'ExtraTree')
g_f = feature_df.nlargest(n, 'GradientBoost')
r_f = feature_df.nlargest(n, 'RandomForest')

result = pd.concat([a_f, e_f, g_f, r_f])
result.drop_duplicates(inplace=True)

result['features'].values

top_features = ['dst_host_srv_diff_host_rate', 'srv_count', 'src_bytes', 'hot',
       'dst_host_diff_srv_rate', 'same_srv_rate', 'dst_host_serror_rate',
       'protocol_type', 'service', 'flag', 'dst_host_same_src_port_rate',
       'count', 'dst_host_srv_count', 'logged_in',
       'dst_host_srv_serror_rate', 'dst_host_same_srv_rate',
       'serror_rate', 'diff_srv_rate', 'srv_serror_rate']

# garbage = np.argsort(result.transpose().mean())
# garbage = garbage.sort_index()[:5] # FML... :'(
# garbage.index

# result = result.drop(garbage.index)
# result.shape

# arr = X_train.columns.to_numpy()
# result = result.set_index(np.take(arr,result.index))

### Remove `selected_features` from our Data

In [ ]:
# X_train_SF = X_train[result.index]
# X_test_SF = X_test[result.index]


print(X_train.shape)


selected_features = result['features'].values.tolist()
X_train_SF = X_train[selected_features]
X_test_SF = X_test[selected_features]



# X_train_SF = X_train_SF[:20000]
# y_train    = y_train[:20000]


print(X_train_SF.shape)
print(y_train.size)

numb_of_train_points = 410000

In [ ]:
DTC = DecisionTreeClassifier()
RFC = RandomForestClassifier(n_estimators=150, random_state=42, n_jobs=-1)
ETC = ExtraTreesClassifier(n_estimators=150, random_state=42, n_jobs=-1)
XGB = xgboost.XGBClassifier(n_estimators=100, n_jobs=-1)
GBM = lightgbm.LGBMClassifier(objective='multiclass', n_estimators= 400)

list_of_CLFs_names = []
list_of_CLFs = [DTC, RFC, ETC, XGB, GBM]
ranking = []

for clf in list_of_CLFs:
    _ = clf.fit(X_train[:numb_of_train_points],y_train[:numb_of_train_points])
    pred = clf.score(X_test,y_test)
    name = str(type(clf)).split(".")[-1][:-2]
    print("Acc: %0.5f for the %s" % (pred, name))

    ranking.append(pred)
    list_of_CLFs_names.append(name)

In [ ]:
DTC = DecisionTreeClassifier()
RFC = RandomForestClassifier(n_estimators=150, random_state=42, n_jobs=-1)
ETC = ExtraTreesClassifier(n_estimators=200, random_state=42, n_jobs=-1)
XGB = xgboost.XGBClassifier(n_estimators=100, n_jobs=-1)
GBM = lightgbm.LGBMClassifier(objective='multiclass', n_estimators= 400)

list_of_CLFs_names = []
list_of_CLFs = [DTC, RFC, ETC, XGB, GBM]
ranking = []

for clf in list_of_CLFs:
    _ = clf.fit(X_train_SF[:numb_of_train_points],y_train[:numb_of_train_points])
    pred = clf.score(X_test_SF,y_test)
    name = str(type(clf)).split(".")[-1][:-2]
    print("Acc: %0.5f for the %s" % (pred, name))

    ranking.append(pred)
    list_of_CLFs_names.append(name)

In [ ]:
from sklearn.decomposition import * 
wanted_explained_variance_ratio = 0.99
steps_down = 4
wanted_n_components = 5
first_time = True

for i in range(X_train.shape[1]-1, 1, -steps_down):
  total_var_ratio = round(np.sum(PCA(n_components=i).fit(X_train).explained_variance_ratio_), 5)
  print('i =', i, 'with a variance ratio of', total_var_ratio)
  if total_var_ratio < wanted_explained_variance_ratio and first_time:
    wanted_n_components = i + steps_down
    first_time = False

print("We should set n_components to: ", wanted_n_components, " is that our default choice?",  first_time)

pca = PCA(n_components=wanted_n_components) # 
_ = pca.fit(X_train)
X_train_PCA = pca.transform(X_train)
X_test_PCA = pca.transform(X_test)

##=================================================================
DTC = DecisionTreeClassifier()
RFC = RandomForestClassifier(n_estimators=150, random_state=42, n_jobs=-1)
ETC = ExtraTreesClassifier(n_estimators=200, random_state=42, n_jobs=-1)
XGB = xgboost.XGBClassifier(n_estimators=100, n_jobs=-1)
GBM = lightgbm.LGBMClassifier(objective='multiclass', n_estimators= 400)

list_of_CLFs_names = []
list_of_CLFs = [DTC, RFC, ETC, XGB, GBM]
ranking = []

for clf in list_of_CLFs:
    _ = clf.fit(X_train_PCA[:numb_of_train_points], y_train[:numb_of_train_points])
    pred = clf.score(X_test_PCA, y_test)
    name = str(type(clf)).split(".")[-1][:-2]
    print("Acc: %0.5f for the %s" % (pred, name))

    ranking.append(pred)
    list_of_CLFs_names.append(name)


## lets multithread this

* https://stackoverflow.com/questions/42920148/using-sklearn-voting-ensemble-with-partial-fit

* https://gist.github.com/tomquisel/a421235422fdf6b51ec2ccc5e3dee1b4



In [ ]:
# import multiprocessing

# ET = ExtraTreesClassifier(n_estimators=10, criterion='gini', max_features='auto', bootstrap=False) # .76 
# RF = RandomForestClassifier(n_estimators=25, random_state=1)# .75
# GB = GradientBoostingClassifier() # .74
# ET = ExtraTreesClassifier(n_estimators=10, criterion='gini', max_features='auto', bootstrap=False) # .77 # without this lil fucker, Acc: 0.75 [Ensemble]
# clfList = [ET,RF,GB,ET]

# def spawn(clf):
#   clf.fit(x,y)
#   print("Done another one!")

# import time
# start = time.time()

# if __name__ == '__main__':
  
#   for i in clfList:
#     print(i)
#     #spawn(i) # 16 secounds 

#     # # 16 secounds
#     # p = threading.Thread(target=spawn, args=(i,))
#     # p.start()
#     # p.join()
    
#   p=multiprocessing.Pool(6) # 15.65
#   results = p.map(spawn,clfList) # clfList has 4 models, first 2 are fast, last 2 are slow
#   results


# end = time.time()
# print(end - start)


In [ ]:
# ET.score(X_test_SF,Y_test)


In [ ]:
# from sklearn.cluster import KMeans
# for i in range(10):
#     tmp = KMeans(i)    
#     tmp = clf.fit(x,y)
#     tmp = tmp.score(X_test_SF,Y_test)
#     print(i,tmp)



nope... that didnt end well. 